<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-18.-Concurrency-with-asyncio" data-toc-modified-id="Chapter-18.-Concurrency-with-asyncio-1">Chapter 18. Concurrency with asyncio</a></span><ul class="toc-item"><li><span><a href="#Spinner-thread-using-threading-module" data-toc-modified-id="Spinner-thread-using-threading-module-1.1">Spinner thread using threading module</a></span></li><li><span><a href="#Spinner-thread-using-asyncio" data-toc-modified-id="Spinner-thread-using-asyncio-1.2">Spinner thread using asyncio</a></span></li><li><span><a href="#Main-differences-between-Thread-and-Task" data-toc-modified-id="Main-differences-between-Thread-and-Task-1.3">Main differences between Thread and Task</a></span><ul class="toc-item"><li><span><a href="#How-the-asyncio.Future-class-differs-from-the-concurrent.futures.Future-class" data-toc-modified-id="How-the-asyncio.Future-class-differs-from-the-concurrent.futures.Future-class-1.3.1">How the asyncio.Future class differs from the concurrent.futures.Future class</a></span></li></ul></li></ul></li></ul></div>

# Chapter 18. Concurrency with asyncio

Parallelism requres multiple cores to do lots of things at once. Concurrency only requires a single core and is about dealing with multiple tasks at once. 


`asyncio` is a package that implements concurrency with coroutines driven by an event loop.

## Spinner thread using threading module

In [1]:
import threading
import itertools
import time
import sys


class Signal:   
    go = True


def spin(msg, signal):   
    for char in itertools.cycle('|/-\\'):   
        status = char + ' ' + msg
        print('\r' + status, end = '')
        print('\r' * len(status), end='')   
        time.sleep(.1)
        if not signal.go:   
            break
    print('\r' + ' ' * (len(status)+2), end='')   


def slow_function():   
    # pretend waiting a long time for I/O
    time.sleep(3)   # GIL released so the secondary thread will proceed.
    return 42


def supervisor():   
    signal = Signal()
    spinner = threading.Thread(target=spin, # secondary thread object created
                               args=('thinking!', signal))
    print('spinner object:', spinner)   
    spinner.start()   # start secondary thread
    result = slow_function()   # run slow function
    signal.go = False          # change state to break for loop
    spinner.join()             # wait until spinner thread finishes
    return result


def main():
    result = supervisor()   
    print('\rAnswer:', result)


if __name__ == '__main__':
    main()


spinner object: <Thread(Thread-4, initial)>
Answer: 42   


## Spinner thread using asyncio

In [2]:
import asyncio
import itertools
import sys
import nest_asyncio
nest_asyncio.apply()

async def spin(msg):  # <1>
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        print('\r' + status, end = '')
        print('\r' * len(status), end='')   
        try:
            await asyncio.sleep(.1)  # sleep without blocking the event loop.
        except asyncio.CancelledError:  # break if asyncio.CancelledError 
                                        # is raised after spin wakes up,
            break
    print('\r' + ' ' * (len(status)+2), end='')   


async def slow_function():  # <5>
    # pretend waiting a long time for I/O
    await asyncio.sleep(3)  # let the event loop proceed while 
                            # this coroutine does I/O by sleeping.
    return 42


async def supervisor():  # <7>
    spinner = asyncio.ensure_future(spin('thinking!'))  # create Task object
#     schedules the spin coroutine to run, wrapping it in a Task object, 
#     which is returned immediately.
    print('spinner Task object:', spinner)  # <9>
    result = await slow_function()  # drive slow_function with await
    spinner.cancel()  # cancel Task object this raises asyncio.CancelledErro
    return result


def main():
    loop = asyncio.get_event_loop()  # create event loop
    result = loop.run_until_complete(supervisor())  # runs supervisor to completion
    print('\rAnswer:', result)


if __name__ == '__main__':
    main()

spinner Task object: <Task pending name='Task-2' coro=<spin() running at <ipython-input-2-317c81792668>:7>>
Answer: 42   


## Main differences between Thread and Task
* asyncio.Task is roughly the equivalent of a threading.Thread
* Task drives a coroutine, and a Thread invokes a callable.
* When you get a Task object, it is already scheduled to run (e.g., by asyncio.async); 
* a Thread instance must be explicitly told to run by calling its start method.
* There’s no API to terminate a thread from the outside, because a thread could be interrupted at any point, leaving the system in an invalid state. 
* Task.cancel() raises CancelledError inside the coroutine. The coroutine can deal with this by catching the exception and breaking
* Instead of holding locks to synchronize the operations of multiple threads, coroutines are “synchronized” by definition: only one of them is running at any time

### How the asyncio.Future class differs from the concurrent.futures.Future class
asyncio.Task is an instance of asyncio.Future because Task is a subclass of Future designed to wrap a coroutine. 
The following methods are common to asyncio.Future and concurrent.futures.Future
* .done(), 
* .add_done_callback(…),
* .results()

In [10]:
import os
import time
import sys
import asyncio

import aiohttp 
import requests   

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()   

BASE_URL = 'http://flupy.org/data/flags'   

DEST_DIR = 'downloads/'   




async def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = await aiohttp.request('GET', url)   
    image = resp.read()   
    return image


async def download_one(cc):   
    image = await get_flag(cc)   
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc


def download_many(cc_list):
    loop = asyncio.get_event_loop()   
    to_do = [download_one(cc) for cc in sorted(cc_list)]   
    wait_coro = await asyncio.wait(to_do)   
    res, _ = loop.run_until_complete(wait_coro)   
#     loop.close()  

    return len(res)

def main(download_many):   
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))

    
main(download_many)


<coroutine object download_many at 0x7fb0844e8ac0> flags downloaded in 0.00s


<ipython-input-10-e9450b41f8e5>:50: RuntimeWarning: coroutine 'download_many' was never awaited
  main(download_many)
